In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar
import matplotlib.pyplot as plt


In [5]:
df=pd.read_csv('data/rheas_model/zmb_n_25_tamsat.csv')
df.rename(columns={'cname':'Region'}, inplace=True)
data=df
data['planting']=pd.to_datetime(data['planting'], format='%Y/%m/%d')
data['harvest'] = pd.to_datetime(data['harvest'], format='%Y/%m/%d')
data['year'] = data['planting'].dt.year
data['pmonth'] = data['planting'].dt.month
data['gwad']=data['gwad']/1000
data1=data[data['year']==2023]
data1
# OND_data=data1.loc[(data1['pmonth']>=10) & (data1['pmonth']<=12) ]
# OND_data1=OND_data.groupby(['Region', 'year'], as_index=False).agg({'gwad': 'mean'})


# MAM_data=data1.loc[(data1['pmonth']>=3) & (data1['pmonth']<=5) ]
# MAM_data1=MAM_data.groupby(['Region', 'year'], as_index=False).agg({'gwad': 'mean'})

# MAM_data['season']='MAM'
# MAM_data
# MAM_data=data1.loc[(data1['hmonth']>2) & (data1['hmonth']<=5) ]
# MAM_data['season']='MAM'
# OND_data['season']='OND'
# OND_data.groupby(['Region', 'season'], as_index=False).agg({'gwad': 'mean'})
# data=data[data['year']==2023]

# 
# 
# MAM_data=data[data['year']==2023]
# MAM_data
# rh_mam23= MAM_data1
# rh_lr23=OND_data1
# rh_lr23
# rh_mam23

# # rh_mam2
# #pd.pivot_table(data, values='gwad', index=[ 'planting', 'harvest'], columns='cname')

Empty DataFrame
Columns: [Region, planting, harvest, lai, wsgd, gwad, year, pmonth]
Index: []

In [302]:
# def format_df(filename):
#     df = pd.read_csv(filename)
#     df['harvest'] = pd.to_datetime(df['harvest'], format='%Y/%m/%d')
#     df['planting'] = pd.to_datetime(df['planting'], format='%Y/%m/%d')
#     df['date'] = df['harvest'].dt.strftime("%Y")
#     df = df.rename(columns={'cname': 'Region'})
#     df['gwad'] =df['gwad']/1000
#     #df = df.rename(columns={df.columns[1]: "Region"})
#     #df['Region'] = df['Region'].str.upper()
#     return df

In [303]:
# data=pd.read_csv('rheas/data/kenya_n_25_tam_yields.csv')
# data['harvest']= pd.to_datetime(data['harvest'], format="%Y/%m/%d" )
# data['gwad']= data['gwad']/1000
# data['planting']= pd.to_datetime(data['planting'], format="%Y/%m/%d" )
# data['pyear']=data['planting'].dt.year
# data.groupby(['cname', 'pyear'], as_index=False).agg({'gwad': 'mean'})




In [304]:
# Season 1:  October--August (LR)
#Generally the maize growing season in Kenya can be grouped into two e.g. 
#March--October and September--February. So we will aggregate the metrics and 
# #forecast with this condition using the function `RH_metrics`.
# lrStart = []
# lrEnd = []
# def LR_metrics(lr, lrStart, lrEnd, season):
#     lr.loc[(pd.to_datetime(lr['planting']).dt.month.isin(lrStart)) & (pd.to_datetime(lr['harvest']).dt.month.isin(lrEnd)), 'Season'] = season
#     lr = lr.groupby(['Region', 'date', 'Season'], as_index=False).agg({'gwad': 'mean'})

#     return lr

# rh_lr = LR_metrics(data, lrStart, lrEnd, "LR")
# print(rh_lr.head())

In [305]:
# # rh_lr23=rh_lr[rh_lr["date"] == '2023'] 
# print(rh_lr23.head())

In [306]:
# # Shortrains season : Sep--Feb
# mamStart = [9,10,11,12,1, 2]
# mamEnd = [12, 1,5, 6, 7, 8]
# def MAM_metrics(mam, mamStart, mamEnd, season):
#     mam.loc[(pd.to_datetime(mam['planting']).dt.month.isin(mamStart)) & (pd.to_datetime(mam['harvest']).dt.month.isin(mamEnd)), 'Season'] = season
#     mam = mam.groupby(['Region', 'date', 'Season'], as_index=False).agg({'gwad': 'mean'})

#     return mam

# rh_mam = MAM_metrics(data, mamStart, mamEnd, "MAM")
# print(rh_mam.head())

In [307]:
# rh_mam23=rh_mam[rh_mam["date"] == '2023'] 
print(rh_mam23.head())

            Region  year      gwad
0          Baringo  2023  3.871475
1            Bomet  2023  2.670975
2          Bungoma  2023  4.035725
3            Busia  2023  4.202475
4  Elgeyo-Marakwet  2023  3.836275


In [308]:
kenya_shp = gpd.read_file("rheas/shapefiles/gadm41_KEN_1.shp")
kenya_shp.rename(columns={'NAME1':'Region'}, inplace=True)
country_shapefile = gpd.read_file("rheas/shapefiles/gadm41_KEN_1.shp")

# shapefile["NewDist20"] = shapefile.index
# print(shapefile)
OND23 = pd.merge(kenya_shp, rh_lr23, on="Region")
MAM23 = pd.merge(kenya_shp, rh_mam23, on="Region")

# Setting yield 'gwad' values over water body to numpy NaN
# LR23.loc[LR23['TYPE_2'] == 'Water body', 'gwad'] = np.NaN
# MAM23.loc[MAM23['TYPE_2'] == 'Water body', 'gwad'] = np.NaN
# def exlude(dataframe):
#     if dataframe['TYPE_2'] == "Wilaya":
#         return dataframe
# exlude(MAM23)
#MAM23.to_csv('rheas/dummy.csv',index=True)


In [309]:
geo_sourceLR23 = GeoJSONDataSource(geojson=OND23.to_json())
geo_sourceMAM23 = GeoJSONDataSource(geojson=MAM23.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

In [310]:
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p1 = figure(title='MAM (Mt/Ha) - 2023',
           outer_width=650, outer_height=600)

p1.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_sourceLR23,
          fill_color={'field': 'gwad', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@Region'), ('gwad', '@gwad')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [311]:
color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p2 = figure(title='MAM (Mt/Ha) - 2023',
           outer_width=650, outer_height=600)

p2.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p2.patches('xs','ys', source=geo_sourceMAM23,
          fill_color={'field': 'gwad', 'transform': color_mapper1},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@Region'), ('Yield', '@gwad')])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p2.add_layout(color_bar, 'right')

In [312]:

# p=gridplot([[p1, p2]], width=650, height=600)
show(p2)

In [313]:
# from bokeh.io import save
# # specify the output file name and location
save(p2, filename='yield_map.html', title='Yield Data Map')

C:\Users\kasit\AppData\Local\Temp\ipykernel_13844\694419692.py:3: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(p2, filename='yield_map.html', title='Yield Data Map')


'd:\\RONO\\RONO1\\majambo\\kenya\\yield_map.html'

In [314]:
#from bokeh.io import save

# specify the output file name and location
#save(p, filename='yield_map.html', title='Yield Data Map')